Librerias pertinentes:

In [12]:
# import librarys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pymongo import MongoClient 
import os
from dotenv import load_dotenv
from pymongo import MongoClient
import configparser
from pprint import pprint as pp
import json

# librarys ML
from sklearn.neural_network import MLPRegressor
import cv2

# para acceder al modelo de alfonso
import pickle as pk

Conexion a la basa de datos, para extraer los Bson de MongoDB:

In [13]:
# Connection and variables to request
#=========== connection to mongoDB ===============#
# Reading credentials from environment
load_dotenv() # esto es para que pueda cargar las variables de entorno
MONGO_USERNAME = os.environ.get("MONGO_USERNAME")
MONGO_PASSWORD = os.environ.get("MONGO_PASSWORD")
DB_Server = os.environ.get("DB_SERVER_IP")

# Loading the configuration
config = configparser.ConfigParser()
config.read('config/satConfig.ini')
annDB = config['DBs']['annotation']

# functions to make connections to MongoDB
def open_DB():
    client = MongoClient(host=DB_Server, port=27017, 
                            username=MONGO_USERNAME, password=MONGO_PASSWORD)
    return(client)

# Opening both database collections and creating the links to the key tables
mongoClient = open_DB()
dbInternal = mongoClient[annDB]
# ==== Key Tables ====
site_annotations = dbInternal['SiteAnnotations']
## ================================================================= ##

In [14]:
# funciones para el calculo del area y el perimetro con cv2
def getArea(Points, PixelSize): # calcula el area de un poligono
    ''' Calculaing the physical area of a Countour'''
    C_array = np.array(Points, dtype=int)
    return(int(cv2.contourArea(C_array)*(PixelSize**2)))


def getPerimeter(Points, PixelSize): # calcula el perimetro, el cual es la suma de sus lados
    ''' Calculating the physical perimeter of a Countour'''
    C_array = np.array(Points, dtype=int)
    return(int(cv2.arcLength(C_array, True)*PixelSize))


In [15]:
# descargamos las annotaciones conn los ides del file "images_info.json"
df_info = pd.read_json(r"config\images_info.json")
annotations = []

for Id, x2, y2 in zip(df_info["id"], df_info["x"], df_info["y"]): 
    
    annotation = site_annotations.find_one({"id":Id})
    x1,y1 = annotation["ImageSize"]
    resolution = annotation["Resolution"]/12
    new_annotation = []
    
    for annot in annotation["Annotations"]:
        
        points = annot["Points"]
        # new_points = [[(point[0]*2494)/2048,(point[1]*2432)/2048] for point in points]
        new_points = [[(point[0]*x2)/x1,(point[1]*y2)/y1] for point in points]
        
        area = getArea(new_points, resolution)
        perimeter = getPerimeter(new_points, resolution)
        
        annot["Attributes"]["area"] = area
        annot["Attributes"]["perimeter"] = perimeter
        annot["Points"] = new_points
        
        new_annotation.append(annot)
    
    annotation["Annotations"] = new_annotation
    annotation["ImageSize"] = [x2,y2]
    annotations.append(annotation)

# exportamos esta data a un json file para probar el modelo de alfonso
df_annot = pd.DataFrame(annotations)

In [16]:
# obtenemos los parametros de los Hps
json = json.load(open("json/labelParamsHP.json", "r"))
classMap2 = json["classMap2"]

Estatimacion de combinaciones para una Regresion lineal (simple o multiple):

In [17]:
"""Estimacion del HP del RTU por su area, el numero de rtu que hay, el area del edificio y RTU, perimetro del RTU"""
data = []
for annotation in annotations:
    id = annotation["id"]
    area_b = int([float(area["Attributes"]["area"]) for area in annotation["Annotations"] if area["Class"] == "Building"][0])
    n_rtu = len(annotation["Annotations"])-1
    for annot in annotation["Annotations"]:
        if annot["Class"] != "Building" and annot["Attributes"]["HP"] != "0.00":
            data.append({
                # "id":id,
                # "address" :  annotation["UsrAddress"],
                "area_b": area_b,
                "n_rtu": n_rtu,
                "area_r":int(annot["Attributes"]["area"]),
                "perimeter": annot["Attributes"]["perimeter"],
                "hp": classMap2[str(float(annot["Attributes"]["HP"]))],
                # "hp":  float(annot["Attributes"]["HP"])
            })

In [18]:
# exportamos el df
df = pd.DataFrame(data)
df.to_csv("excel/data.csv", index=False)
df.head()

,area_b,n_rtu,area_r,perimeter,hp
0,11094,7,26,20,1
1,11094,7,27,21,1
2,11094,7,16,16,0
3,11094,7,19,17,0
4,11094,7,15,15,0
